In [1]:
import random
import ollama

print("🔁 양방향 코드브레이커 - 당신과 AI가 서로의 코드를 추리합니다!")

# 사용자 코드 입력 (중복 없이 5자리)
while True:
    player_code = input("👤 당신의 5자리 비밀 코드를 설정하세요 (0~9, 첫 자리는 0 제외, 중복 없이): ").strip()
    if player_code.isdigit() and len(player_code) == 5 and len(set(player_code)) == 5 and player_code[0] != '0':
        break
    print("❗ 중복 없는 5자리 숫자를 정확히 입력하세요.")

# AI 코드 생성 (중복 없이 5자리)
first_digit = random.choice("123456789")
ai_code = first_digit + "".join(random.sample([d for d in "0123456789" if d != first_digit], 4))

# 초기 설정
user_attempts = 0
ai_attempts = 0
ai_history = ""
user_won = False
ai_won = False
turn_order = random.choice(["user", "ai"])
print(f"🕹️ 선턴 결정: {'사용자' if turn_order == 'user' else 'AI'}가 먼저 추리합니다.")

def get_ai_guess(history):
    prompt = f"""
[게임 룰]
- 당신과 사용자는 각각 중복 없는 5자리 숫자 코드를 설정합니다 (첫 숫자는 0이 될 수 없음)
- 양쪽은 번갈아가며 서로의 코드를 추리합니다
- 피드백은 '스트라이크(숫자+위치 일치)', '볼(숫자만 일치)', '아웃(불일치)'로 주어집니다
- 선턴이 먼저 정답을 맞힌 경우에만 상대에게 단 한 번의 반격 기회가 주어집니다

당신은 5자리 숫자 코드브레이커 AI입니다. 숫자는 0~9로 이루어지며 중복되지 않습니다.
첫 숫자는 0이 될 수 없습니다.
지금까지의 추측과 피드백을 참고하여 가장 가능성 높은 5자리 숫자를 한 줄로 출력하세요.
지금까지의 기록:
{history}
예시 형식 (형식만 참고, 이 숫자를 그대로 쓰지 마세요): 38429
"""
    for _ in range(5):
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=[{"role": "user", "content": prompt}])
        guess = ''.join(filter(str.isdigit, response["message"]["content"].strip()))
        if len(guess) == 5 and len(set(guess)) == 5 and guess[0] != '0':
            return guess
    return "12345"

def format_feedback(exact, partial):
    if exact == 0 and partial == 0:
        return "아웃"
    return f"{exact} 스트라이크, {partial} 볼"

# 게임 루프 시작
turn_sequence = ["user", "ai"] if turn_order == "user" else ["ai", "user"]
t = 0
while True:
    print()
    print(f"🔄 턴 {t + 1}")

    for role in turn_sequence:
        if role == "user":
            while True:
                user_guess = input("👤 당신의 추측 (5자리 숫자 입력): ").strip()
                if user_guess.isdigit() and len(user_guess) == 5:
                    break
                print("❗ 5자리 숫자를 정확히 입력하세요. 예: 12345")

            user_attempts += 1
            exact = sum(user_guess[i] == ai_code[i] for i in range(5))
            partial = sum(min(user_guess.count(d), ai_code.count(d)) for d in set(user_guess)) - exact
            print(f"👤 당신의 추측 결과 → {format_feedback(exact, partial)}")

            if user_guess == ai_code:
                user_won = True
                print(f"🏆 당신이 AI의 코드를 맞췄습니다! 🎉 (총 시도: {user_attempts})")
                break

        else:
            ai_guess = get_ai_guess(ai_history)
            ai_attempts += 1
            exact = sum(ai_guess[i] == player_code[i] for i in range(5))
            partial = sum(min(ai_guess.count(d), player_code.count(d)) for d in set(ai_guess)) - exact
            ai_history += f"{ai_guess} → {format_feedback(exact, partial)}"
            print(f"🤖 AI의 추측: {ai_guess} → {format_feedback(exact, partial)}")

            if ai_guess == player_code:
                ai_won = True
                print(f"🤖 AI가 당신의 코드를 맞췄습니다! (총 시도: {ai_attempts})")
                break

    if user_won or ai_won:
        break
    t += 1

# 반격 처리
if user_won:
    ai_guess = get_ai_guess(ai_history)
    ai_attempts += 1
    exact = sum(ai_guess[i] == player_code[i] for i in range(5))
    partial = sum(min(ai_guess.count(d), player_code.count(d)) for d in set(ai_guess)) - exact
    print(f"🤖 (반격) AI의 마지막 추측: {ai_guess} → {format_feedback(exact, partial)}")
    if ai_guess == player_code:
        print("🤝 AI가 반격에 성공했습니다. 게임은 무승부입니다!")
    else:
        print("🏆 당신의 승리입니다! AI는 반격에 실패했습니다.")
elif ai_won:
    while True:
        user_guess = input("👤 (반격) 당신의 마지막 추측 (5자리 숫자 입력): ").strip()
        if user_guess.isdigit() and len(user_guess) == 5:
            break
        print("❗ 5자리 숫자를 정확히 입력하세요. 예: 12345")

    user_attempts += 1
    exact = sum(user_guess[i] == ai_code[i] for i in range(5))
    partial = sum(min(user_guess.count(d), ai_code.count(d)) for d in set(user_guess)) - exact
    print(f"👤 (반격) 당신의 추측 결과 → {format_feedback(exact, partial)}")
    if user_guess == ai_code:
        print("🤝 당신이 반격에 성공했습니다. 게임은 무승부입니다!")
    else:
        print("🏆 AI의 승리입니다! 당신은 반격에 실패했습니다.")

ModuleNotFoundError: No module named 'ollama'

In [2]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 243.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 32.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 148.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 117.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 251.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [14]:
import random
import gradio as gr
import ollama

# 상태 초기화 함수
def initialize():
    first_digit = random.choice("123456789")
    ai_code = first_digit + "".join(random.sample([d for d in "0123456789" if d != first_digit], 4))
    history = ""
    return "", ai_code, history, 0, 0, random.choice(["user", "ai"]), False, False

def format_feedback(exact, partial):
    if exact == 0 and partial == 0:
        return "아웃"
    return f"{exact} 스트라이크, {partial} 볼"

def get_ai_guess(history):
    prompt = f"""
[게임 룰]
- 당신과 사용자는 각각 중복 없는 5자리 숫자 코드를 설정합니다 (첫 숫자는 0이 될 수 없음)
- 양쪽은 번갈아가며 서로의 코드를 추리합니다
- 피드백은 '스트라이크(숫자+위치 일치)', '볼(숫자만 일치)', '아웃(불일치)'로 주어집니다
- 선턴이 먼저 정답을 맞힌 경우에만 상대에게 단 한 번의 반격 기회가 주어집니다

당신은 5자리 숫자 코드브레이커 AI입니다. 숫자는 0~9로 이루어지며 중복되지 않습니다.
첫 숫자는 0이 될 수 없습니다.
지금까지의 추측과 피드백을 참고하여 가장 가능성 높은 5자리 숫자를 한 줄로 출력하세요.
지금까지의 기록:
{history}
예시 형식 (형식만 참고, 이 숫자를 그대로 쓰지 마세요): 38429
"""
    for _ in range(5):
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=[{"role": "user", "content": prompt}])
        guess = ''.join(filter(str.isdigit, response["message"]["content"].strip()))
        if len(guess) == 5 and len(set(guess)) == 5 and guess[0] != '0':
            return guess
    return "12345"

def process_turn(user_guess, user_code, ai_code, history, user_attempts, ai_attempts, turn_order, user_won, ai_won):
    result = history

    if user_code.strip() == "":
        return "❗ 먼저 비밀 코드를 입력하고 시작해주세요.", user_code, ai_code, history, user_attempts, ai_attempts, turn_order, user_won, ai_won, gr.update(interactive=True)
    if not user_code.isdigit() or len(user_code) != 5 or user_code[0] == '0' or len(set(user_code)) != 5:
        return "❗ 비밀 코드는 중복 없는 5자리 숫자이며 첫 숫자는 0이 될 수 없습니다.", user_code, ai_code, history, user_attempts, ai_attempts, turn_order, user_won, ai_won, gr.update(interactive=True)

    # 사용자 코드 입력 완료 후 비활성화
    user_code_display = gr.update(interactive=False)

    turn_sequence = ["user", "ai"] if turn_order == "user" else ["ai", "user"]

    for role in turn_sequence:
        if role == "user":
            if not user_guess or not user_guess.isdigit() or len(user_guess) != 5:
                return "❗ 5자리 숫자를 정확히 입력하세요. 예: 12345", user_code, ai_code, history, user_attempts, ai_attempts, turn_order, user_won, ai_won, user_code_display
            user_attempts = int(user_attempts) + 1
            exact = sum(user_guess[i] == ai_code[i] for i in range(5))
            partial = sum(min(user_guess.count(d), ai_code.count(d)) for d in set(user_guess)) - exact
            history += f"👤 {user_guess} → {format_feedback(exact, partial)}\n"
            result += f"👤 {user_guess} → {format_feedback(exact, partial)}\n"
            if user_guess == ai_code:
                result += f"🏆 당신이 AI의 코드를 맞췄습니다! 🎉 (총 시도: {user_attempts})"
                user_won = True
                break
        else:
            ai_guess = get_ai_guess(history)
            ai_attempts = int(ai_attempts) + 1
            exact = sum(ai_guess[i] == user_code[i] for i in range(5))
            partial = sum(min(ai_guess.count(d), user_code.count(d)) for d in set(ai_guess)) - exact
            history += f"{ai_guess} → {format_feedback(exact, partial)}\n"
            result += f"🤖 {ai_guess} → {format_feedback(exact, partial)}\n"
            if ai_guess == user_code:
                result += f"🤖 AI가 당신의 코드를 맞췄습니다! (총 시도: {ai_attempts})"
                ai_won = True
                break

    return result.strip(), user_code, ai_code, history, user_attempts, ai_attempts, turn_order, user_won, ai_won, user_code_display

def wrapped_process_turn(*args):
    *outputs, user_code_display = process_turn(*args)
    return (*outputs, user_code_display, gr.update(value=""))

def ui():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 코드브레이커: 사용자 vs AI")

        user_code = gr.Textbox(label="🔐 당신의 비밀 코드 (중복 없는 5자리, 첫 숫자 0 불가)", interactive=True, lines=1)
        user_guess = gr.Textbox(label="🎯 이번 턴의 추측 (5자리 숫자)", lines=1, interactive=True, autofocus=True, placeholder="예: 12345", show_label=True)
        output_box = gr.Markdown(label="📋 결과 출력")
        next_btn = gr.Button("🔄 턴 진행", variant="primary")
        reset_btn = gr.Button("🔁 게임 초기화")

        initial = initialize()
        state = [gr.State(v) for v in initial[1:]]  # user_code 제외 상태 저장

        next_btn.click(fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[output_box, user_code, *state, user_code, user_guess]
        )

        user_guess.submit(
            fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[output_box, user_code, *state, user_code, user_guess]
        )

        reset_btn.click(
            fn=lambda: ("", *initialize()[1:], gr.update(interactive=True), gr.update(value="")),
            inputs=[],
            outputs=[user_code, *state, user_guess]
        )

    return demo

ui().launch(share=True)


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://73d83905642d1938a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1855: UserWarning: A function returned too many output values (needed: 9, returned: 10). Ignoring extra values.
    Output components:
        [textbox, state, state, state, state, state, state, state, textbox]
    Output values returned:
        ["", "73156", "", 0, 0, "user", False, False, {'interactive': True, '__type__': 'update'}, {'value': '', '__type__': 'update'}]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1855: UserWarning: A function returned too many output values (needed: 9, returned: 10). Ignoring extra values.
    Output components:
        [textbox, state, state, state, state, state, state, state, textbox]
    Output values returned:
        ["", "47328", "", 0, 0, "ai", False, False, {'interactive': True, '__type__': 'update'}, {'value': '', '__type__': 'update'}]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1855: UserWarning: A function returned too many

In [30]:
import random
import gradio as gr
import ollama
from dataclasses import dataclass

# 💬 상수 메시지 정의
MSG_REQUIRE_CODE = "❗ 먼저 비밀 코드를 입력하고 시작해주세요."
MSG_INVALID_CODE = "❗ 비밀 코드는 중복 없는 5자리 숫자이며 첫 숫자는 0이 될 수 없습니다."
MSG_INVALID_GUESS = "❗ 5자리 숫자를 정확히 입력하세요. 예: 12345"

# 🧠 게임 상태 데이터 구조
@dataclass
class GameState:
    ai_code: str
    history: str
    user_attempts: int
    ai_attempts: int
    turn_order: str
    user_won: bool
    ai_won: bool

# 🔁 초기화 함수
def initialize():
    first_digit = random.choice("123456789")
    ai_code = first_digit + "".join(random.sample([d for d in "0123456789" if d != first_digit], 4))
    return GameState(
        ai_code=ai_code,
        history="",
        user_attempts=0,
        ai_attempts=0,
        turn_order=random.choice(["user", "ai"]),
        user_won=False,
        ai_won=False,
    )

def format_feedback(exact, partial):
    if exact == 0 and partial == 0:
        return "아웃"
    return f"{exact} 스트라이크, {partial} 볼"

def validate_code_input(code):
    return code.isdigit() and len(code) == 5 and code[0] != '0' and len(set(code)) == 5

def validate_guess_input(guess):
    return guess.isdigit() and len(guess) == 5

def compare_codes(guess, target):
    exact = sum(guess[i] == target[i] for i in range(5))
    partial = sum(min(guess.count(d), target.count(d)) for d in set(guess)) - exact
    return exact, partial

def get_ai_guess(history):
    prompt = f"""
[게임 룰]
- 당신과 사용자는 각각 중복 없는 5자리 숫자 코드를 설정합니다 (첫 숫자는 0이 될 수 없음)
- 양쪽은 번갈아가며 서로의 코드를 추리합니다
- 피드백은 '스트라이크(숫자+위치 일치)', '볼(숫자만 일치)', '아웃(불일치)'로 주어집니다
- 선턴이 먼저 정답을 맞힌 경우에만 상대에게 단 한 번의 반격 기회가 주어집니다

당신은 5자리 숫자 코드브레이커 AI입니다. 숫자는 0~9로 이루어지며 중복되지 않습니다.
첫 숫자는 0이 될 수 없습니다.
지금까지의 추측과 피드백을 참고하여 가장 가능성 높은 5자리 숫자를 한 줄로 출력하세요.
지금까지의 기록:
{history}
예시 형식 (형식만 참고, 이 숫자를 그대로 쓰지 마세요): 38429
"""
    for _ in range(5):
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=[{"role": "user", "content": prompt}])
        guess = ''.join(filter(str.isdigit, response["message"]["content"].strip()))
        if validate_code_input(guess):
            return guess
    return "12345"

def process_turn(user_guess, user_code, state: GameState):
    result = state.history
    user_code_display = gr.update(interactive=False)

    if user_code.strip() == "":
        return MSG_REQUIRE_CODE, user_code, state, user_code_display
    if not validate_code_input(user_code):
        return MSG_INVALID_CODE, user_code, state, user_code_display

    turn_sequence = ["user", "ai"] if state.turn_order == "user" else ["ai", "user"]

    for role in turn_sequence:
        if role == "user":
            if not validate_guess_input(user_guess):
                return MSG_INVALID_GUESS, user_code, state, user_code_display
            state.user_attempts += 1
            turn_number = state.user_attempts
            exact, partial = compare_codes(user_guess, state.ai_code)
            feedback = format_feedback(exact, partial)
            line = f"[TURN {turn_number}] 👤 사용자: {user_guess} → {feedback}\n"
            state.history += "\n" + line
            result += "\n" + line
            if user_guess == state.ai_code:
                result += f"🏆 당신이 AI의 코드를 맞췄습니다! 🎉 (총 시도: {state.user_attempts})"
                state.user_won = True
                break
        else:
            ai_guess = get_ai_guess(state.history)
            state.ai_attempts += 1
            turn_number = state.ai_attempts
            exact, partial = compare_codes(ai_guess, user_code)
            feedback = format_feedback(exact, partial)
            line = f"[TURN {turn_number}] 🤖 AI: {ai_guess} → {feedback}\n"
            state.history += "\n" + line
            result += "\n" + line
            if ai_guess == user_code:
                result += f"🤖 AI가 당신의 코드를 맞췄습니다! (총 시도: {state.ai_attempts})"
                state.ai_won = True
                break

    return result.strip(), user_code, state, user_code_display

def wrapped_process_turn(user_guess, user_code, *state_vars):
    state = GameState(*state_vars)
    result, user_code, updated_state, user_code_display = process_turn(user_guess, user_code, state)
    return result, user_code, *updated_state.__dict__.values(), user_code_display, gr.update(value="")

def reset_game():
    state = initialize()
    return "", *state.__dict__.values(), gr.update(interactive=True), gr.update(value="")

def ui():
    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 코드브레이커: 사용자 vs AI")

        user_code = gr.Textbox(label="🔐 당신의 비밀 코드 (중복 없는 5자리, 첫 숫자 0 불가)", interactive=True, lines=1)
        user_guess = gr.Textbox(label="🎯 이번 턴의 추측 (5자리 숫자)", lines=1, interactive=True, autofocus=True, placeholder="예: 12345", show_label=True)
        output_box = gr.Markdown(label="📋 결과 출력")
        next_btn = gr.Button("🔄 턴 진행", variant="primary")
        reset_btn = gr.Button("🔁 게임 초기화")

        initial_state = initialize()
        state = [gr.State(v) for v in initial_state.__dict__.values()]

        next_btn.click(fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[output_box, user_code, *state, user_code, user_guess]
        )

        user_guess.submit(
            fn=wrapped_process_turn,
            inputs=[user_guess, user_code, *state],
            outputs=[output_box, user_code, *state, user_code, user_guess]
        )

        reset_btn.click(
            fn=reset_game,
            inputs=[],
            outputs=[user_code, *state, user_code, user_guess]
        )

    return demo

ui().launch(share=True)


* Running on local URL:  http://127.0.0.1:7877
* Running on public URL: https://ce96d0681795557a14.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
